In [ ]:
import pandas as pd
import numpy as np
import os
import numpy
import matplotlib.pyplot as plt
import SimpleITK
import itertools
import sys
import torch
from torchvision import transforms
from PIL import Image
from matplotlib import cm
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchvision import models

from pathlib import Path

SOURCE_PATH = Path(os.getcwd()) / "src"

if SOURCE_PATH not in sys.path:
    sys.path.append(SOURCE_PATH)

from src.extraction import (
    extract_images_in_survival_order,
)

from src.plots import (
    plot_observation,
    plot_deeplab_mobile_predictions,
    plot_mobile_prediction_from_path,
)

from src.deeplab_mobile.modelling import (
    select_images_input,
    get_deeplab_mobile_model,
    train_deeplab_mobile,
)

from src.deeplab_mobile.segdataset import get_mobile_dataloaders

from src.utils import LOGS_FILE_PATH, TYPE_NAMES

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = Path(os.getcwd()) / "data" / "HGG"
survival = pd.read_csv(data_path.parent / "survival_data.csv")

In [ ]:
dir_ids = survival["BraTS19ID"]

In [ ]:
t2, t1ce, t1, flair, seg = extract_images_in_survival_order(data_path, dir_ids)
images = [t2, t1ce, t1, flair, seg]

In [ ]:
for i in range(len(TYPE_NAMES)):
    survival[TYPE_NAMES[i]] = images[i]

In [ ]:
modelname = "flair_totalpipe_old.pt"
model = torch.load(Path(os.getcwd()) / "models" / modelname)
model.eval()

In [ ]:
flair_segout = []

for i in range(len(survival)):
    input_tensor = (
        torch.tensor(survival["flair"][i])
        .expand(3, -1, -1)
        .type(torch.ShortTensor)
        .float()
        .unsqueeze(0)
    )
    with torch.no_grad():
        output = model(input_tensor)["out"][0][0]
    flair_segout.append(np.array(output))

survival["flair_seg"] = flair_segout

In [ ]:
survival = survival.dropna().reset_index(drop=True)
survival["Survival"] = survival["Survival"].apply(lambda x: str(x))

In [ ]:
to_drop = []
for i in range(len(survival)):
    if "ALIVE" in survival["Survival"].loc[i]:
        to_drop.append(i)


survival.drop(to_drop, inplace=True)
print(len(survival))

In [ ]:
survival["Survival"] = survival["Survival"].apply(lambda x: int(x))

In [ ]:
from tensorflow import keras

In [ ]:
def build_model(input_img, age):
    base_model = keras.applications.resnet50.ResNet50(
        weights="imagenet", include_top=False, input_tensor=input_img
    )
    a = keras.layers.GlobalAveragePooling2D()(base_model.output)
    a = keras.layers.concatenate([a, age])
    a = keras.layers.BatchNormalization()(a)
    output = keras.layers.Dense(1)(a)

    model = keras.models.Model(inputs=[input_img, age], outputs=output)
    return model, base_model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(51984, 1200)
        self.fc2 = nn.Linear(1200, 340)
        self.fc3 = nn.Linear(340, 30)
        self.fc4 = nn.Linear(30, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


net = Net()

In [ ]:
from torch.utils.data import Dataset


class Survival_Dataset(Dataset):
    def __init__(
        self, images, targets, fraction=0.1, subset=None, image_color_mode="rgb"
    ) -> None:

        if image_color_mode not in ["rgb", "grayscale"]:
            raise ValueError(
                f"{image_color_mode} is an invalid choice. Please enter from rgb grayscale."
            )

        self.image_color_mode = image_color_mode

        if subset not in ["Train", "Test"]:
            raise (
                ValueError(
                    f"{subset} is not a valid input. Acceptable values are Train and Test."
                )
            )

        self.fraction = fraction
        self.images = images
        self.targets = targets

    def __len__(self) -> int:
        return len(self.images)

    def __getitem__(self, index: int):
        image = self.images[index]
        target = self.targets[index]

        sample = {"image": image, "target": target}
        sample["image"] = (
            torch.tensor(sample["image"]).expand(3, -1, -1).type(torch.ShortTensor)
        )
        sample["target"] = target
        return sample

In [ ]:
def get_survival_dataloaders(images, target, batch_size=14):
    image_datasets = {
        x: Survival_Dataset(images, target, subset=x) for x in ["Train", "Test"]
    }

    dataloaders = {
        x: DataLoader(
            image_datasets[x],
            batch_size=batch_size,
        )
        for x in ["Train", "Test"]
    }
    return dataloaders

In [ ]:
dataloaders = get_survival_dataloaders(
    survival["flair"].values, survival["Survival"].values
)

In [ ]:
import time
import copy
from sklearn.metrics import f1_score
from tqdm import tqdm

num_epochs = 2
metrics = {"f1_score": f1_score}
running_loss = 0.0
best_loss = 1e10

since = time.time()
best_model_wts = copy.deepcopy(net.state_dict())


fieldnames = (
    ["epoch", "Train_loss", "Test_loss"]
    + [f"Train_{m}" for m in metrics.keys()]
    + [f"Test_{m}" for m in metrics.keys()]
)

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, num_epochs + 1):
    print("Epoch {}/{}".format(epoch, num_epochs))
    print("-" * 10)

    batchsummary = {a: [0] for a in fieldnames}

    for phase in ["Train", "Test"]:
        if phase == "Train":
            net.train()
        else:
            net.eval()

        for sample in tqdm(iter(dataloaders[phase])):
            inputs = sample["image"].float()
            targets = sample["target"].float()
            # zero the parameter gradients
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == "Train"):
                outputs = net(inputs).squeeze()
                print(outputs.squeeze().shape)
                print(targets.shape)
                loss = criterion(outputs, targets)
                print(loss)
                y_pred = outputs.data.cpu().numpy().ravel()
                y_true = targets.data.cpu().numpy().ravel()
                for name, metric in metrics.items():
                    if name == "f1_score":
                        # Use a classification threshold of 0.1
                        batchsummary[f"{phase}_{name}"].append(
                            metric(y_true > 0, y_pred > 0.1)
                        )
                    else:
                        batchsummary[f"{phase}_{name}"].append(
                            metric(y_true.astype("uint8"), y_pred)
                        )

                if phase == "Train":
                    loss.backward()
                    optimizer.step()
        batchsummary["epoch"] = epoch
        epoch_loss = loss
        batchsummary[f"{phase}_loss"] = epoch_loss.item()
        print("{} Loss: {:.4f}".format(phase, loss))
    for field in fieldnames[3:]:
        batchsummary[field] = np.mean(batchsummary[field])
        print(batchsummary)
        # deep copy the model
        if phase == "Test" and loss < best_loss:
            best_loss = loss
            best_model_wts = copy.deepcopy(net.state_dict())

time_elapsed = time.time() - since
print(
    "Training complete in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60)
)
print("Lowest Loss: {:4f}".format(best_loss))

# load best model weights
net.load_state_dict(best_model_wts)

In [ ]:
net()

In [ ]:
survival